# Logistic Regression
Data file: https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/credit_card_churners_1_10k.csv

## Requirements
* Load data into dataframe
* Examine data
* Use SimpleImputer to replace missing values
* Prepare data for model training
* Train Logistic Regression model (change hyperparameters and re-train as needed)
* Test model and evaluate model performance metrics

In [1]:
from datetime import datetime
print(f'Run time: {datetime.now().strftime("%D %T")}')

Run time: 02/13/24 15:00:48


### Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

### Load data

#### Credit Card Churn Prediction
* https://www.kaggle.com/datasets/anwarsan/credit-card-bank-churn

Business Problem  
A business manager of a consumer credit card bank is facing the problem of customer attrition. They want to analyze the data to find out the reason behind this and leverage the same to predict customers who are likely to drop off.

Columns
* CLIENTNUM: Client number. Unique identifier for the customer holding the account
* Attrition_Flag: Internal event (customer activity) variable - if the account is closed then "Attrited Customer" else "Existing Customer"
* Customer_Age: Age in Years
* Gender: Gender of the account holder
* Dependent_count: Number of dependents
* Education_Level: Educational Qualification of the account holder - High School, College, Post-Graduate
* Marital_Status: Marital Status of the account holder
* Income_Category: Annual Income Category of the account holder
* Card_Category: Type of Card
* Months_on_book: Period of relationship with the bank
* Total_Relationship_Count: Total no. of products held by the customer
* Months_Inactive_12_mon: No. of months inactive in the last 12 months
* Contacts_Count_12_mon: No. of Contacts between the customer and bank in the last 12 months
* Credit_Limit: Credit Limit on the Credit Card
* Total_Revolving_Bal: The balance that carries over from one month to the next is the revolving balance
* Avg_Open_To_Buy: Open to Buy refers to the amount left on the credit card to use (Average of last 12 months)
* Total_Trans_Amt: Total Transaction Amount (Last 12 months)
* Total_Trans_Ct: Total Transaction Count (Last 12 months)
* Total_Ct_Chng_Q4_Q1: Ratio of the total transaction count in 4th quarter and the total transaction count in 1st quarter
* Total_Amt_Chng_Q4_Q1: Ratio of the total transaction amount in 4th quarter and the total transaction amount in 1st quarter
* Avg_Utilization_Ratio: Represents how much of the available credit the customer spent

In [3]:
# Read data from file (credit_card_churners_1_10k.csv) into dataframe
#  NOTE: CLIENTNUM as the index column
df = pd.read_csv("https://raw.githubusercontent.com/vjavaly/Baruch-CIS-STA-3920/main/data/credit_card_churners_1_10k.csv", index_col='CLIENTNUM')

### Examine data

In [4]:
# Review dataframe shape
df.shape

(10000, 23)

In [5]:
# Display first few rows of dataframe
df.head()

,Attrition_Flag,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,...,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender_F,Gender_M,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single
CLIENTNUM,,,,,,,,,,,,,,,,,,,,,
712965183,1,63.0,2,1.0,0.0,0.0,52,5,2,3,...,0.416,1188,35,0.750,0.781,1,0,0,1,0
714225333,1,48.0,4,1.0,0.0,0.0,36,5,1,1,...,0.661,1545,21,0.909,0.264,1,0,0,1,0
710512833,1,38.0,2,1.0,0.0,0.0,29,6,1,1,...,0.615,5178,79,0.756,0.405,1,0,0,1,0
716396358,1,52.0,2,1.0,1.0,0.0,47,5,3,0,...,0.921,1531,35,0.667,0.619,0,1,0,1,0
715609533,0,47.0,3,0.0,0.0,0.0,35,1,3,3,...,0.621,1887,36,0.333,0.000,1,0,0,0,1


In [6]:
# Display distribution counts for target variable Attrition_Flag
df['Attrition_Flag'].value_counts()

Attrition_Flag
1    8392
0    1608
Name: count, dtype: int64

### Prepare data

##### Check for missing values

In [7]:
df.isnull().sum()

Attrition_Flag                0
Customer_Age                502
Dependent_count               0
Education_Level               0
Income_Category               0
Card_Category                 0
Months_on_book                0
Total_Relationship_Count      0
Months_Inactive_12_mon        0
Contacts_Count_12_mon         0
Credit_Limit                  0
Total_Revolving_Bal           0
Avg_Open_To_Buy               0
Total_Amt_Chng_Q4_Q1          0
Total_Trans_Amt               0
Total_Trans_Ct                0
Total_Ct_Chng_Q4_Q1           0
Avg_Utilization_Ratio         0
Gender_F                      0
Gender_M                      0
Marital_Status_Divorced       0
Marital_Status_Married        0
Marital_Status_Single         0
dtype: int64

#### Use the SimpleImputer to replace missing values

In [8]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [9]:
df = pd.DataFrame(imp_mean.fit_transform(df), columns=df.columns)

#### Check for missing values again

In [10]:
df.isnull().sum()

Attrition_Flag              0
Customer_Age                0
Dependent_count             0
Education_Level             0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
Gender_F                    0
Gender_M                    0
Marital_Status_Divorced     0
Marital_Status_Married      0
Marital_Status_Single       0
dtype: int64

### Separate independent and dependent variables
* Independent variables: All remaining variables except Attrition_Flag
* Dependent variable: Attrition_Flag

In [11]:
X = df.drop("Attrition_Flag", axis = 1)
y = df["Attrition_Flag"]

### Split data into training and test sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

### Train Logistic Regression model

In [13]:
model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

### If above results in error, review error message, look up documentation for LogisticRegression, and change model hyperparameter appropriately

### Test model

In [14]:
# Generate predictions against the test set
predictions = model.predict(X_test)

### Model evaluation

In [15]:
# Print model accuracy
accuracy = model.score(X_test, y_test)
print("accuracy = ", round((accuracy * 100), 5), "%")

accuracy =  88.83333 %


In [16]:
# Print classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.75      0.47      0.58       488
         1.0       0.90      0.97      0.94      2512

    accuracy                           0.89      3000
   macro avg       0.83      0.72      0.76      3000
weighted avg       0.88      0.89      0.88      3000



In [17]:
# Print confusion matrix
confusion_matrix(y_test, predictions)

array([[ 230,  258],
       [  77, 2435]])